<a href="https://colab.research.google.com/github/matheusrdr/dataanalyst-renner-test-resolution/blob/main/TesteRenner_DataAnalyst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importando bibliotecas:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files

In [2]:
# importando datasets:
df_raw = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/upload/Exercicio_Python_Merge_Summarizado.xlsx', sheet_name="Raw")
df_produto = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/upload/Exercicio_Python_Merge_Summarizado.xlsx', sheet_name="Master Produto")
df_lojas = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/upload/Exercicio_Python_Merge_Summarizado.xlsx', sheet_name="Master Lojas")
df_clientes = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/upload/Exercicio_Python_Merge_Summarizado.xlsx', sheet_name="Master clientes")

In [3]:
# Renomeando as colunas dos dataframes:
df_raw.columns = ['CodigoTransacao', 'Data', 'LojaID', 'ProdID', 'ClienteID', 'UnidadesVendidas', 'Faturamento', 'Promocao']
df_produto.columns = ['ProdID', 'Descricao', 'Categoria', 'Tamanho', 'Cor']
df_lojas.columns = ['LojaID', 'CodigoCidade', 'UF', 'Regiao']
df_clientes.columns = ['ClienteID', 'Genero', 'FaixadeIdade', 'ClasseSocial']

In [4]:
# Eliminando espacos em branco:
df_produto['Categoria'] = df_produto['Categoria'].str.strip()
df_produto['Tamanho'] = df_produto['Tamanho'].str.strip()
df_produto['Cor'] = df_produto['Cor'].str.strip()

In [5]:
# Criando a descrição do produto como a concatenação da categoria_Cor_Tamanho:
df_produto['Descricao'] = df_produto['Categoria'] + df_produto['Cor'] + df_produto['Tamanho'] 

In [6]:
# Criando df_cidades para alimentar demais datasets:
df_cidades = pd.DataFrame({'CodigoCidade' : df_lojas['CodigoCidade'], 'UF' : df_lojas['UF'], 'Regiao' : df_lojas['Regiao'] }).drop_duplicates().dropna().reset_index(drop = True)

In [7]:
# Completando informações do df_lojas:

# Função para excluir colunas pós merge (terminadas com '_x'):
def drop_x(df):
    to_drop = [x for x in df_lojas if x.endswith('_x')]
    df_lojas.drop(to_drop, axis=1, inplace = True)

# Completando informações e deixando somente as colunas com registros mergeados:
df_lojas = pd.merge(df_lojas, df_cidades, on = ['CodigoCidade'], how = 'left', suffixes=('_x', ''))
drop_x(df_lojas)


In [8]:
# Trazendo informações dos dataframes para o df_raw:

df_raw = pd.merge(df_raw, df_produto, on='ProdID', how='left')
df_raw = pd.merge(df_raw, df_lojas, on='LojaID', how='left')
df_raw = pd.merge(df_raw, df_clientes, on='ClienteID', how='left')
#df_raw.head(10)

In [9]:
# Adicionando a informação de Ano no df_raw:

df_raw['Ano'] = df_raw['Data'].dt.year
#df_raw.head(5)

In [10]:
# Separando o df_raw em 2 df, por ano:

df_raw_2022 = df_raw.loc[df_raw['Ano'] == 2022]
df_raw_2023 = df_raw.loc[df_raw['Ano'] == 2023]

In [11]:
# Agrupando os df_raw criados:

df_raw_2022 = df_raw_2022.groupby(['ProdID', 'Descricao']).agg({'CodigoTransacao': 'count', 'ClienteID': 'nunique', 'Faturamento': 'sum', 'UnidadesVendidas': 'sum'})
df_raw_2023 = df_raw_2023.groupby(['ProdID', 'Descricao']).agg({'CodigoTransacao': 'count', 'ClienteID': 'nunique', 'Faturamento': 'sum', 'UnidadesVendidas': 'sum'})

In [12]:
# Criando a tabela base para cálculo dos KPI's, unindo os dois agrupamentos por ano criados, renomeando as colunas para facilitar a identificação:

df_resumo_1 = pd.concat([df_raw_2022, df_raw_2023], axis=1)
df_resumo_1.columns = ['CodigoTransacao_2022', 'ClienteID_2022', 'Faturamento_2022', 'UnidadesVendidas_2022', 'CodigoTransacao_2023', 'ClienteID_2023', 'Faturamento_2023', 'UnidadesVendidas_2023']
#df_resumo_1

In [13]:
# Criando colunas com indicadores:

# Substituindo valores faltantes por 0:
df_resumo_1.fillna(value=0, inplace=True)

## dataframes auxiliares:
df_clientes = df_raw.groupby(['ProdID', 'Descricao']).agg({'ClienteID': 'nunique'})
df_frequencia = df_raw.groupby(['ProdID', 'Descricao']).agg({'CodigoTransacao': 'nunique'})
df_transacao = df_raw.groupby(['ProdID', 'Descricao']).agg({'CodigoTransacao': 'nunique'})

## KPIs:
df_resumo_1['Vendas'] = df_resumo_1['CodigoTransacao_2022'] + df_resumo_1['CodigoTransacao_2023']
df_resumo_1['VariacaoVendas'] = (((df_resumo_1['CodigoTransacao_2023'] - df_resumo_1['CodigoTransacao_2022']) / df_resumo_1['CodigoTransacao_2022']) * 100).round(2)
df_resumo_1['VarClientes'] = (((df_resumo_1['ClienteID_2023'] - df_resumo_1['ClienteID_2022']) / df_resumo_1['ClienteID_2022']) * 100).round(2)
df_resumo_1['GastoPorCliente'] = ((df_resumo_1['Faturamento_2023'] + df_resumo_1['Faturamento_2022']) / df_clientes['ClienteID']).round(2)
df_resumo_1['VarGastoPorCliente'] = ((((df_resumo_1['Faturamento_2023'] / df_resumo_1['ClienteID_2023']) - (df_resumo_1['Faturamento_2022'] / df_resumo_1['ClienteID_2022'])) / (df_resumo_1['Faturamento_2022'] / df_resumo_1['ClienteID_2022'])) * 100).round(2)
df_resumo_1['FrequenciaMedia'] = (df_frequencia['CodigoTransacao'] / df_clientes['ClienteID']).round(2)
df_resumo_1['VarFrequenciaMedia'] = (((df_resumo_1['CodigoTransacao_2023'] / df_resumo_1['ClienteID_2023']) - (df_resumo_1['CodigoTransacao_2022'] / df_resumo_1['ClienteID_2022'])) / (df_resumo_1['CodigoTransacao_2022'] / df_resumo_1['ClienteID_2022'])).round(2) 
df_resumo_1['TicketMedio'] = ((df_resumo_1['Faturamento_2023'] + df_resumo_1['Faturamento_2022']) / df_transacao['CodigoTransacao']).round(2)
df_resumo_1['VarTicketMedio'] = ((((df_resumo_1['Faturamento_2023'] / df_resumo_1['CodigoTransacao_2023']) - (df_resumo_1['Faturamento_2022'] / df_resumo_1['CodigoTransacao_2022']) ) / ((df_resumo_1['Faturamento_2022'] / df_resumo_1['CodigoTransacao_2022']))) * 100).round(2)
df_resumo_1['PeçasPorCupom'] = ((df_resumo_1['UnidadesVendidas_2023'] + df_resumo_1['UnidadesVendidas_2022']) / df_transacao['CodigoTransacao']).round(2)
df_resumo_1['VarPecasPorCupom'] = ((((df_resumo_1['UnidadesVendidas_2023'] / df_resumo_1['CodigoTransacao_2023']) - (df_resumo_1['UnidadesVendidas_2022'] / df_resumo_1['CodigoTransacao_2022'])) / (df_resumo_1['UnidadesVendidas_2022'] / df_resumo_1['CodigoTransacao_2022'])) * 100).round(2)
df_resumo_1['PrecoMedio'] = ((df_resumo_1['Faturamento_2023'] + df_resumo_1['Faturamento_2022']) / (df_resumo_1['UnidadesVendidas_2023'] + df_resumo_1['UnidadesVendidas_2022'])).round(2)
df_resumo_1['VarPrecoMedio'] = ((((df_resumo_1['Faturamento_2023'] / df_resumo_1['UnidadesVendidas_2023']) - (df_resumo_1['Faturamento_2022'] / df_resumo_1['UnidadesVendidas_2022'])) / (df_resumo_1['Faturamento_2022'] / df_resumo_1['UnidadesVendidas_2022'])) * 100).round(2)


In [17]:
# Gerando tabela final:
df_resumo_1_exportar = df_resumo_1.iloc[:,8:23]
#df_resumo_1_exportar:
df_resumo_1_exportar.to_excel('df_resumo_1.xlsx') 
files.download('df_resumo_1.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>